<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# Artificial Intelligence in Finance

## Machine Learning

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

## Learning

<blockquote>"A computer program is said to learn from experience 𝐸 with respect to some class of tasks 𝑇 and performance measure 𝑃, if its performance at tasks in 𝑇, as measured by 𝑃, improves with experience 𝐸." — Mitchell (1997)</blockquote>


## Data

In [ ]:
!git clone https://github.com/tpq-classes/ai_in_finance_book.git
import sys
sys.path.append('ai_in_finance_book')


In [ ]:
!pip install -U scikit-learn==1.5.2

In [ ]:
#!pip install scikeras

In [ ]:
import numpy as np
import pandas as pd
from pylab import plt, mpl
np.random.seed(100)
plt.style.use('seaborn-v0_8')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'

In [ ]:
url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True)['EUR=']

In [ ]:
raw.head()

In [ ]:
raw.tail()

In [ ]:
l = raw.resample('1M').last()

In [ ]:
l.tail()

In [ ]:
l.plot(figsize=(10, 6), title='EUR/USD monthly');

In [ ]:
l = l.values
l -= l.mean()

In [ ]:
f = np.linspace(-2, 2, len(l))

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(f, l, 'ro')
plt.title('Sample Data Set')
plt.xlabel('features')
plt.ylabel('labels');

## Success

In [ ]:
def MSE(l, p):
    return np.mean((l - p) ** 2)

In [ ]:
reg = np.polyfit(f, l, deg=5)
reg

In [ ]:
p = np.polyval(reg, f)

In [ ]:
MSE(l, p)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(f, l, 'ro', label='sample data')
plt.plot(f, p, '--', label='regression')
plt.legend();

In [ ]:
%%time
for i in range(10, len(f) + 1, 20):
    reg = np.polyfit(f[:i], l[:i], deg=3)
    p = np.polyval(reg, f)
    mse = MSE(l, p)
    print(f'{i:3d} | MSE={mse}')

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '6'

In [ ]:
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(100)

In [ ]:
from keras.layers import Dense
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=1))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='rmsprop')

In [ ]:
model.summary()

In [ ]:
%time hist = model.fit(f, l, epochs=1500, verbose=False)

In [ ]:
import tensorflow as tf
p = model(tf.convert_to_tensor(f, dtype=tf.float32), training=False).numpy().flatten()

In [ ]:
MSE(l, p)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(f, l, 'ro', label='sample data')
plt.plot(f, p, '--', label='DNN approximation')
plt.legend();

In [ ]:
import pandas as pd

In [ ]:
res = pd.DataFrame(hist.history)

In [ ]:
res.tail()

In [ ]:
res.iloc[100:].plot(figsize=(10, 6))
plt.ylabel('MSE')
plt.xlabel('epochs');

## Capacity

In [ ]:
reg = {}
for d in range(1, 12, 2):
    reg[d] = np.polyfit(f, l, deg=d)
    p = np.polyval(reg[d], f)
    mse = MSE(l, p)
    print(f'{d:2d} | MSE={mse}')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(f, l, 'ro', label='sample data')
for d in reg:
    p = np.polyval(reg[d], f)
    plt.plot(f, p, '--', label=f'deg={d}')
plt.legend();

In [ ]:
def create_dnn_model(hl=1, hu=256):
    ''' Function to create Keras DNN model.

    Parameters
    ==========
    hl: int
        number of hidden layers
    hu: int
        number of hidden units (per layer)
    '''
    model = Sequential()
    for _ in range(hl):
        model.add(Dense(hu, activation='relu', input_dim=1))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='rmsprop')
    return model

In [ ]:
model = create_dnn_model(3)

In [ ]:
model.summary()

In [ ]:
%time model.fit(f, l, epochs=2500, verbose=False)

In [ ]:
import tensorflow as tf
p = model(tf.convert_to_tensor(f, dtype=tf.float32), training=False).numpy().flatten()

In [ ]:
MSE(l, p)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(f, l, 'r', label='sample data')
plt.plot(f, p, '--', label='DNN approximation')
plt.legend();

## Evaluation

In [ ]:
te = int(0.25 * len(f))
va = int(0.25 * len(f))

In [ ]:
np.random.seed(100)
ind = np.arange(len(f))
np.random.shuffle(ind)

In [ ]:
ind_te = np.sort(ind[:te])
ind_va = np.sort(ind[te:te + va])
ind_tr = np.sort(ind[te + va:])

In [ ]:
f_te = f[ind_te]
f_va = f[ind_va]
f_tr = f[ind_tr]

In [ ]:
l_te = l[ind_te]
l_va = l[ind_va]
l_tr = l[ind_tr]

In [ ]:
reg = {}
mse = {}
for d in range(1, 22, 4):
    reg[d] = np.polyfit(f_tr, l_tr, deg=d)
    p = np.polyval(reg[d], f_tr)
    mse_tr = MSE(l_tr, p)
    p = np.polyval(reg[d], f_va)
    mse_va = MSE(l_va, p)
    mse[d] = (mse_tr, mse_va)
    print(f'{d:2d} | MSE_tr={mse_tr:7.5f} | MSE_va={mse_va:7.5f}')

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
ax[0].plot(f_tr, l_tr, 'ro', label='training data')
ax[1].plot(f_va, l_va, 'go', label='validation data')
for d in reg:
    p = np.polyval(reg[d], f_tr)
    ax[0].plot(f_tr, p, '--', label=f'deg={d} (tr)')
    p = np.polyval(reg[d], f_va)
    plt.plot(f_va, p, '--', label=f'deg={d} (va)')
ax[0].legend()
ax[1].legend();

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
model = create_dnn_model(2, 256)

In [ ]:
callbacks = [EarlyStopping(monitor='loss',
                           patience=100,
                          restore_best_weights=True)]

In [ ]:
%%time
hist = model.fit(f_tr, l_tr, epochs=3000, verbose=False,
          validation_data=(f_va, l_va),
          callbacks=callbacks)

In [ ]:
import tensorflow as tf
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(10, 8))
ax[0].plot(f_tr, l_tr, 'ro', label='training data')
p = model(tf.convert_to_tensor(f_tr, dtype=tf.float32), training=False).numpy()
ax[0].plot(f_tr, p, '--', label=f'DNN (tr)')
ax[0].legend()
ax[1].plot(f_va, l_va, 'go', label='validation data')
p = model(tf.convert_to_tensor(f_va, dtype=tf.float32), training=False).numpy()
ax[1].plot(f_va, p, '--', label=f'DNN (va)')
ax[1].legend();

In [ ]:
res = pd.DataFrame(hist.history)

In [ ]:
res.tail()

In [ ]:
res.iloc[35::25].plot(figsize=(10, 6))
plt.ylabel('MSE')
plt.xlabel('epochs');

In [ ]:
import tensorflow as tf
p_ols = np.polyval(reg[5], f_te)
p_dnn = model(tf.convert_to_tensor(f_te, dtype=tf.float32), training=False).numpy().flatten()

In [ ]:
MSE(l_te, p_ols)

In [ ]:
MSE(l_te, p_dnn)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(f_te, l_te, 'ro', label='test data')
plt.plot(f_te, p_ols, '--', label='OLS prediction')
plt.plot(f_te, p_dnn, '-.', label='DNN prediction');
plt.legend();

## Bias & Variance

In [ ]:
f_tr = f[:20:2]
l_tr = l[:20:2]

In [ ]:
f_va = f[1:20:2]
l_va = l[1:20:2]

In [ ]:
reg_b = np.polyfit(f_tr, l_tr, deg=1)

In [ ]:
reg_v = np.polyfit(f_tr, l_tr, deg=9, full=True)[0]

In [ ]:
f_ = np.linspace(f_tr.min(), f_va.max(), 75)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(f_tr, l_tr, 'ro', label='training data')
plt.plot(f_va, l_va, 'go', label='validation data')
plt.plot(f_, np.polyval(reg_b, f_), '--', label='high bias')
plt.plot(f_, np.polyval(reg_v, f_), '--', label='high variance')
plt.ylim(-0.2)
plt.legend(loc=2);

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
def evaluate(reg, f, l):
    p = np.polyval(reg, f)
    bias = np.abs(l - p).mean()
    var = p.var()
    msg = f'MSE={MSE(l, p):.4f} | R2={r2_score(l, p):9.4f} | '
    msg += f'bias={bias:.4f} | var={var:.4f}'
    print(msg)

In [ ]:
evaluate(reg_b, f_tr, l_tr)

In [ ]:
evaluate(reg_b, f_va, l_va)

In [ ]:
evaluate(reg_v, f_tr, l_tr)

In [ ]:
evaluate(reg_v, f_va, l_va)

## Cross-Validation

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

In [ ]:
def PolynomialRegression(degree=None, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                        LinearRegression(**kwargs))

In [ ]:
np.set_printoptions(suppress=True,
        formatter={'float': lambda x: f'{x:12.2f}'})

In [ ]:
print('\nCross-validation scores')
print(74 * '=')
for deg in range(0, 10, 1):
    model = PolynomialRegression(deg)
    cvs = cross_val_score(model, f.reshape(-1, 1), l, cv=5)
    print(f'deg={deg} | ' + str(cvs.round(2)))

In [ ]:
np.random.seed(100)
tf.random.set_seed(100)
#from keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasRegressor

In [ ]:
model=create_dnn_model
model = KerasRegressor(model=model,
                      verbose=False, epochs=1000,
                      hl=1, hu=36)

In [ ]:
%time cross_val_score(model, f.reshape(-1, 1), l, cv=5)

In [ ]:
help(KerasRegressor)

In [ ]:
model = KerasRegressor(model=create_dnn_model,
                      verbose=False, epochs=1000,
                      hl=3, hu=256)

In [ ]:
%time cross_val_score(model, f.reshape(-1, 1), l, cv=5)

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

<br><br><br><a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:ai@tpq.io">ai@tpq.io</a>